In [430]:
%matplotlib inline
import matplotlib.pyplot as plt

import requests
import json

import os
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

import boto3

from IPython.display import HTML, display
import datetime

#key: XwVHPFh2zHGkoQZ3QmLtDqyncNjhdloY07lr9868
key_defined = "XwVHPFh2zHGkoQZ3QmLtDqyncNjhdloY07lr9868"

In [18]:
nasa = requests.get("https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-07&end_date=2023-09-08&api_key=XwVHPFh2zHGkoQZ3QmLtDqyncNjhdloY07lr9868")

In [19]:
nasa.json()

{'links': {'next': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-08&end_date=2023-09-09&detailed=false&api_key=XwVHPFh2zHGkoQZ3QmLtDqyncNjhdloY07lr9868',
  'previous': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-06&end_date=2023-09-07&detailed=false&api_key=XwVHPFh2zHGkoQZ3QmLtDqyncNjhdloY07lr9868',
  'self': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-07&end_date=2023-09-08&detailed=false&api_key=XwVHPFh2zHGkoQZ3QmLtDqyncNjhdloY07lr9868'},
 'element_count': 58,
 'near_earth_objects': {'2023-09-08': [{'links': {'self': 'http://api.nasa.gov/neo/rest/v1/neo/2277617?api_key=XwVHPFh2zHGkoQZ3QmLtDqyncNjhdloY07lr9868'},
    'id': '2277617',
    'neo_reference_id': '2277617',
    'name': '277617 (2006 BT7)',
    'nasa_jpl_url': 'https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=2277617',
    'absolute_magnitude_h': 18.18,
    'estimated_diameter': {'kilometers': {'estimated_diameter_min': 0.6145468212,
      'estimated_diameter_max': 1.3741684676},
  

In [29]:
import datetime

def iterate_over_dates(start_date_str, end_date_str):
    try:
        # Convert start and end dates to datetime objects
        start_date = datetime.datetime.strptime(start_date_str, "%Y-%m-%d")
        end_date = datetime.datetime.strptime(end_date_str, "%Y-%m-%d")
        
        date_strings = []
        
        # Iterate over each day between start and end dates
        while start_date <= end_date:
            date_strings.append(start_date.strftime("%Y-%m-%d"))
            
            
            start_date += datetime.timedelta(days=1)  # Increment the date by one day
            
        return date_strings
            
    except Exception as e:
        return str(e)


In [428]:
def request_nasa(start_date_str, end_date_str):
    template_url = "https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key={key}"
    try:
        request_url = template_url.format(
            start_date=start_date_str,
            end_date=end_date_str,
            key=key_defined)
        r = requests.get(request_url)
        return r.json()
    except Exception as e:
        print(e)


In [431]:
request_nasa("2023-08-20","2023-08-26")

{'links': {'next': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2023-08-26&end_date=2023-09-01&detailed=false&api_key=XwVHPFh2zHGkoQZ3QmLtDqyncNjhdloY07lr9868',
  'previous': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2023-08-14&end_date=2023-08-20&detailed=false&api_key=XwVHPFh2zHGkoQZ3QmLtDqyncNjhdloY07lr9868',
  'self': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2023-08-20&end_date=2023-08-26&detailed=false&api_key=XwVHPFh2zHGkoQZ3QmLtDqyncNjhdloY07lr9868'},
 'element_count': 124,
 'near_earth_objects': {'2023-08-25': [{'links': {'self': 'http://api.nasa.gov/neo/rest/v1/neo/2101955?api_key=XwVHPFh2zHGkoQZ3QmLtDqyncNjhdloY07lr9868'},
    'id': '2101955',
    'neo_reference_id': '2101955',
    'name': '101955 Bennu (1999 RQ36)',
    'nasa_jpl_url': 'https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=2101955',
    'absolute_magnitude_h': 20.21,
    'estimated_diameter': {'kilometers': {'estimated_diameter_min': 0.2412986969,
      'estimated_diameter_max': 0.5395602

In [436]:
days_from_period = iterate_over_dates("2023-08-20","2023-08-28")

In [437]:
days_from_period

['2023-08-20',
 '2023-08-21',
 '2023-08-22',
 '2023-08-23',
 '2023-08-24',
 '2023-08-25',
 '2023-08-26',
 '2023-08-27',
 '2023-08-28']

In [439]:
def find_date_after(days_after, input_date_str, date_list):
    if input_date_str not in date_list:
        return "Input date is not in the list."
    
    input_index = date_list.index(input_date_str)
    target_index = min(input_index + days_after, len(date_list) - 1)
    
    return date_list[target_index]




In [440]:
def iterate_dates(start_date_str):
    end_date_str = find_date_after(6,start_date_str,days_from_period)
    start_index = days_from_period.index(start_date_str)
    end_index = days_from_period.index(end_date_str)
    return days_from_period[start_index:end_index + 1]



In [442]:

def seven_days(start_d):
    end_date=(find_date_after(6,start_d,days_from_period))
    week = iterate_dates(start_d)
    raw1=pd.DataFrame()

    for day in week:
        neo_data = pd.DataFrame((request_nasa(start_d,end_date)["near_earth_objects"][day]))
        df = pd.DataFrame(neo_data)
        df.insert(1, 'date', day)
        raw1 = pd.concat([raw1, df], ignore_index=True)
    return(raw1)

        

In [443]:
raw = pd.DataFrame()
for d in days_from_period[::7]:
    df1 = ((seven_days(d)))
    raw = pd.concat([raw, df1], ignore_index=True)
raw

,links,date,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,estimated_diameter,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,sentry_data
0,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/...,2023-08-20,3329278,3329278,(2006 GB),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,20.34,{'kilometers': {'estimated_diameter_min': 0.22...,True,"[{'close_approach_date': '2023-08-20', 'close_...",False,NaN
1,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/...,2023-08-20,3527906,3527906,(2010 LH14),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,22.08,{'kilometers': {'estimated_diameter_min': 0.10...,False,"[{'close_approach_date': '2023-08-20', 'close_...",False,NaN
2,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/...,2023-08-20,3726791,3726791,(2015 RH2),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,22.01,{'kilometers': {'estimated_diameter_min': 0.10...,True,"[{'close_approach_date': '2023-08-20', 'close_...",False,NaN
3,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/...,2023-08-20,3755501,3755501,(2016 ND39),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,25.20,{'kilometers': {'estimated_diameter_min': 0.02...,False,"[{'close_approach_date': '2023-08-20', 'close_...",False,NaN
4,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/...,2023-08-20,3760649,3760649,(2016 TY18),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,23.80,{'kilometers': {'estimated_diameter_min': 0.04...,False,"[{'close_approach_date': '2023-08-20', 'close_...",False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
170,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/...,2023-08-28,54317205,54317205,(2022 UE3),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,24.61,{'kilometers': {'estimated_diameter_min': 0.03...,False,"[{'close_approach_date': '2023-08-28', 'close_...",False,NaN
171,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/...,2023-08-28,54351261,54351261,(2023 FX2),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,24.01,{'kilometers': {'estimated_diameter_min': 0.04...,False,"[{'close_approach_date': '2023-08-28', 'close_...",False,NaN
172,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/...,2023-08-28,54382911,54382911,(2023 RT9),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,25.07,{'kilometers': {'estimated_diameter_min': 0.02...,False,"[{'close_approach_date': '2023-08-28', 'close_...",False,NaN
173,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/...,2023-08-28,54384211,54384211,(2023 RQ16),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,26.18,{'kilometers': {'estimated_diameter_min': 0.01...,False,"[{'close_approach_date': '2023-08-28', 'close_...",False,NaN


In [401]:
raw.columns

Index(['links', 'date', 'id', 'neo_reference_id', 'name', 'nasa_jpl_url',
       'absolute_magnitude_h', 'estimated_diameter',
       'is_potentially_hazardous_asteroid', 'close_approach_data',
       'is_sentry_object', 'sentry_data'],
      dtype='object')

In [402]:
dia = raw["estimated_diameter"]
close = raw['close_approach_data']
close

0      [{'close_approach_date': '2023-08-25', 'close_...
1      [{'close_approach_date': '2023-08-25', 'close_...
2      [{'close_approach_date': '2023-08-25', 'close_...
3      [{'close_approach_date': '2023-08-25', 'close_...
4      [{'close_approach_date': '2023-08-25', 'close_...
                             ...                        
959    [{'close_approach_date': '2023-09-27', 'close_...
960    [{'close_approach_date': '2023-09-27', 'close_...
961    [{'close_approach_date': '2023-09-27', 'close_...
962    [{'close_approach_date': '2023-09-27', 'close_...
963    [{'close_approach_date': '2023-09-27', 'close_...
Name: close_approach_data, Length: 964, dtype: object

In [403]:
dia_list = dia.tolist()
dia_units_df = pd.json_normalize(dia_list)

In [404]:
col_index_1 = raw.columns.get_loc("estimated_diameter")
left = raw.iloc[:, :col_index_1]
right = raw.iloc[:, col_index_1:]

In [405]:
result_df = pd.concat([left, dia_units_df, right], axis=1)
result_df = result_df.drop('estimated_diameter', axis=1)
result_df

,links,date,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,kilometers.estimated_diameter_min,kilometers.estimated_diameter_max,meters.estimated_diameter_min,meters.estimated_diameter_max,miles.estimated_diameter_min,miles.estimated_diameter_max,feet.estimated_diameter_min,feet.estimated_diameter_max,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,sentry_data
0,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/...,2023-08-25,2101955,2101955,101955 Bennu (1999 RQ36),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,20.21,0.241299,0.539560,241.298697,539.560289,0.149936,0.335267,791.662417,1770.210979,True,"[{'close_approach_date': '2023-08-25', 'close_...",True,http://api.nasa.gov/neo/rest/v1/neo/sentry/210...
1,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/...,2023-08-25,3158115,3158115,(2003 QK5),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,22.85,0.071541,0.159971,71.540995,159.970528,0.044453,0.099401,234.714558,524.837708,False,"[{'close_approach_date': '2023-08-25', 'close_...",False,NaN
2,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/...,2023-08-25,3248972,3248972,(2004 OW10),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,24.61,0.031809,0.071128,31.809363,71.127899,0.019765,0.044197,104.361432,233.359255,False,"[{'close_approach_date': '2023-08-25', 'close_...",False,NaN
3,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/...,2023-08-25,3408586,3408586,(2008 GS3),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,22.27,0.093445,0.208949,93.444765,208.948847,0.058064,0.129835,306.577323,685.527735,False,"[{'close_approach_date': '2023-08-25', 'close_...",False,NaN
4,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/...,2023-08-25,3602525,3602525,(2012 GA5),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,24.19,0.038597,0.086306,38.597127,86.305799,0.023983,0.053628,126.630997,283.155518,False,"[{'close_approach_date': '2023-08-25', 'close_...",False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/...,2023-09-27,54392059,54392059,(2023 TC3),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,26.59,0.012781,0.028579,12.780710,28.578536,0.007942,0.017758,41.931464,93.761604,False,"[{'close_approach_date': '2023-09-27', 'close_...",False,NaN
960,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/...,2023-09-27,54395879,54395879,(2023 TU26),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,23.95,0.043108,0.096392,43.107712,96.391775,0.026786,0.059895,141.429507,316.245992,False,"[{'close_approach_date': '2023-09-27', 'close_...",False,NaN
961,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/...,2023-09-27,54397457,54397457,(2023 UT3),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,22.62,0.079534,0.177844,79.534394,177.844312,0.049420,0.110507,260.939621,583.478731,False,"[{'close_approach_date': '2023-09-27', 'close_...",False,NaN
962,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/...,2023-09-27,54406289,54406289,(2023 UX27),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,25.91,0.017481,0.039088,17.480545,39.087688,0.010862,0.024288,57.350872,128.240449,False,"[{'close_approach_date': '2023-09-27', 'close_...",True,http://api.nasa.gov/neo/rest/v1/neo/sentry/544...


In [406]:
close_df = pd.DataFrame([x[0] for x in close])
close_df
vel = close_df['relative_velocity']
vel_list = vel.tolist()
vel_units_df = pd.json_normalize(vel_list)
miss = close_df['miss_distance']
miss_list = miss.tolist()
miss_units_df = pd.json_normalize(miss_list)


In [407]:
close_df = close_df.drop(['relative_velocity', 'miss_distance'], axis=1)
close_df

,close_approach_date,close_approach_date_full,epoch_date_close_approach,orbiting_body
0,2023-08-25,2023-Aug-25 06:51,1692946260000,Earth
1,2023-08-25,2023-Aug-25 10:16,1692958560000,Earth
2,2023-08-25,2023-Aug-25 21:29,1692998940000,Earth
3,2023-08-25,2023-Aug-25 13:48,1692971280000,Earth
4,2023-08-25,2023-Aug-25 08:41,1692952860000,Earth
...,...,...,...,...
959,2023-09-27,2023-Sep-27 14:46,1695825960000,Earth
960,2023-09-27,2023-Sep-27 11:22,1695813720000,Earth
961,2023-09-27,2023-Sep-27 04:51,1695790260000,Earth
962,2023-09-27,2023-Sep-27 16:11,1695831060000,Earth


In [408]:
col_index_close = result_df.columns.get_loc("close_approach_data")
result_df = result_df.drop('close_approach_data', axis=1)
left_close = result_df.iloc[:, :col_index_close]
right_close = result_df.iloc[:, col_index_close:]
final_df= pd.concat([left_close, close_df, vel_units_df, miss_units_df , right_close], axis=1)
final_df

,links,date,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,kilometers.estimated_diameter_min,kilometers.estimated_diameter_max,meters.estimated_diameter_min,...,orbiting_body,kilometers_per_second,kilometers_per_hour,miles_per_hour,astronomical,lunar,kilometers,miles,is_sentry_object,sentry_data
0,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/...,2023-08-25,2101955,2101955,101955 Bennu (1999 RQ36),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,20.21,0.241299,0.539560,241.298697,...,Earth,19.9688592017,71887.8931261816,44668.341148321,0.4967706275,193.2437740975,74315827.752563425,46177714.076388265,True,http://api.nasa.gov/neo/rest/v1/neo/sentry/210...
1,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/...,2023-08-25,3158115,3158115,(2003 QK5),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,22.85,0.071541,0.159971,71.540995,...,Earth,8.881183946,31972.2622056877,19866.3203688619,0.2937237596,114.2585424844,43940448.804552052,27303328.8149682376,False,NaN
2,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/...,2023-08-25,3248972,3248972,(2004 OW10),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,24.61,0.031809,0.071128,31.809363,...,Earth,4.177301952,15038.2870270498,9344.2067363355,0.2962462802,115.2398029978,44317812.513343174,27537811.7504928412,False,NaN
3,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/...,2023-08-25,3408586,3408586,(2008 GS3),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,22.27,0.093445,0.208949,93.444765,...,Earth,21.9081117957,78869.2024646521,49006.2552758834,0.4957086352,192.8306590928,74156955.966527024,46078995.7261759712,False,NaN
4,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/...,2023-08-25,3602525,3602525,(2012 GA5),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,24.19,0.038597,0.086306,38.597127,...,Earth,5.1097672483,18395.1620939486,11430.0383577649,0.1578803064,61.4154391896,23618557.552387368,14675891.1329389584,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/...,2023-09-27,54392059,54392059,(2023 TC3),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,26.59,0.012781,0.028579,12.780710,...,Earth,11.1536590879,40153.1727164926,24949.6200137562,0.0393648826,15.3129393314,5888902.589760062,3659194.3901783756,False,NaN
960,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/...,2023-09-27,54395879,54395879,(2023 TU26),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,23.95,0.043108,0.096392,43.107712,...,Earth,14.5989337094,52556.1613539487,32656.3548146244,0.0864523541,33.6299657449,12933088.029845767,8036248.2559629046,False,NaN
961,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/...,2023-09-27,54397457,54397457,(2023 UT3),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,22.62,0.079534,0.177844,79.534394,...,Earth,12.4497756649,44819.1923937404,27848.9031848772,0.1765070985,68.6612613165,26405085.975480195,16407359.605788891,False,NaN
962,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/...,2023-09-27,54406289,54406289,(2023 UX27),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,25.91,0.017481,0.039088,17.480545,...,Earth,9.4321036586,33955.5731711088,21098.6726740148,0.0288524976,11.2236215664,4316272.185140112,2682007.1695210656,True,http://api.nasa.gov/neo/rest/v1/neo/sentry/544...


In [409]:
column_names_mapping = {'kilometers_per_second': 'relative_velocity_km/s', 'kilometers_per_hour': 'relative_velocity_km/h','miles_per_hour': 'relative_velocity_m/h',"astronomical": "miss_dist_astromnomical","lunar": "miss_dist_lunar","kilometers": "miss_dist_km","miles": "miss_dist_miles"}


In [410]:
final_df = final_df.rename(columns=column_names_mapping)


In [411]:
final_df.head().style

,links,date,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,kilometers.estimated_diameter_min,kilometers.estimated_diameter_max,meters.estimated_diameter_min,meters.estimated_diameter_max,miles.estimated_diameter_min,miles.estimated_diameter_max,feet.estimated_diameter_min,feet.estimated_diameter_max,is_potentially_hazardous_asteroid,close_approach_date,close_approach_date_full,epoch_date_close_approach,orbiting_body,relative_velocity_km/s,relative_velocity_km/h,relative_velocity_m/h,miss_dist_astromnomical,miss_dist_lunar,miss_dist_km,miss_dist_miles,is_sentry_object,sentry_data
0,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/2101955?api_key=XwVHPFh2zHGkoQZ3QmLtDqyncNjhdloY07lr9868'},2023-08-25,2101955,2101955,101955 Bennu (1999 RQ36),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=2101955,20.210000,0.241299,0.539560,241.298697,539.560289,0.149936,0.335267,791.662417,1770.210979,True,2023-08-25,2023-Aug-25 06:51,1692946260000,Earth,19.9688592017,71887.8931261816,44668.341148321,0.4967706275,193.2437740975,74315827.752563425,46177714.076388265,True,http://api.nasa.gov/neo/rest/v1/neo/sentry/2101955?api_key=XwVHPFh2zHGkoQZ3QmLtDqyncNjhdloY07lr9868
1,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/3158115?api_key=XwVHPFh2zHGkoQZ3QmLtDqyncNjhdloY07lr9868'},2023-08-25,3158115,3158115,(2003 QK5),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=3158115,22.850000,0.071541,0.159971,71.540995,159.970528,0.044453,0.099401,234.714558,524.837708,False,2023-08-25,2023-Aug-25 10:16,1692958560000,Earth,8.881183946,31972.2622056877,19866.3203688619,0.2937237596,114.2585424844,43940448.804552052,27303328.8149682376,False,nan
2,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/3248972?api_key=XwVHPFh2zHGkoQZ3QmLtDqyncNjhdloY07lr9868'},2023-08-25,3248972,3248972,(2004 OW10),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=3248972,24.610000,0.031809,0.071128,31.809363,71.127899,0.019765,0.044197,104.361432,233.359255,False,2023-08-25,2023-Aug-25 21:29,1692998940000,Earth,4.177301952,15038.2870270498,9344.2067363355,0.2962462802,115.2398029978,44317812.513343174,27537811.7504928412,False,nan
3,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/3408586?api_key=XwVHPFh2zHGkoQZ3QmLtDqyncNjhdloY07lr9868'},2023-08-25,3408586,3408586,(2008 GS3),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=3408586,22.270000,0.093445,0.208949,93.444765,208.948847,0.058064,0.129835,306.577323,685.527735,False,2023-08-25,2023-Aug-25 13:48,1692971280000,Earth,21.9081117957,78869.2024646521,49006.2552758834,0.4957086352,192.8306590928,74156955.966527024,46078995.7261759712,False,nan
4,{'self': 'http://api.nasa.gov/neo/rest/v1/neo/3602525?api_key=XwVHPFh2zHGkoQZ3QmLtDqyncNjhdloY07lr9868'},2023-08-25,3602525,3602525,(2012 GA5),https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=3602525,24.190000,0.038597,0.086306,38.597127,86.305799,0.023983,0.053628,126.630997,283.155518,False,2023-08-25,2023-Aug-25 08:41,1692952860000,Earth,5.1097672483,18395.1620939486,11430.0383577649,0.1578803064,61.4154391896,23618557.552387368,14675891.1329389584,False,nan


In [412]:
# Save final_df to CSV
final_df.to_csv('final_df.csv', index=False)
